This notebook demonstrates using vector index to sample. There are different ways to determine the uncertainty: 
min_dist_diff: if true, prioritize the sentences that have distances to two centroids have smaller difference
                if false, then prioritize the sentences that have a smaller difference of the max differences to all centroids (max distance- min distance).  
This notebook use  min_dist_diff=False           

# Import 

In [1]:
import sklearn_crfsuite
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite import CRF
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from spacy.lang.en import English
from spacy import displacy
from pathlib import Path
import joblib
from spacy.tokens import Doc
from typing import List
import random
from statistics import mean, stdev
from loguru import logger
import sys
import spacy

In [2]:
from medspacy_io.reader.brat_reader import BratDocReader, BratDirReader
import medspacy

## Initiate BratDirReader

In [3]:
cleaned_train_dir=r'..\data\n2c2\cleaned_training'
cleaned_test_dir=r'..\data\n2c2\cleaned_test'
Path(cleaned_train_dir).exists(), Path(cleaned_test_dir).exists()

(True, True)

In [4]:
nlp=spacy.load('en_core_web_sm', disable=['ner'])

In [5]:
dir_reader = BratDirReader(nlp=nlp, schema_file=str(Path(cleaned_train_dir, 'annotation.conf')), support_overlap=True)

## Read eHOST annotations | load from pickles

In [6]:
pickle_file= r'..\data\n2c2\spacy_docs.joblib'

In [7]:
if not Path(pickle_file).exists():
    train_docs=dir_reader.read(txt_dir=cleaned_train_dir)
    test_docs=dir_reader.read(txt_dir=cleaned_test_dir)
    print(len(train_docs), len(test_docs))
    joblib.dump((train_docs, test_docs), pickle_file)
else:
    print(f'{pickle_file} already exists, load them directly')
    # before load from pickle, initiate EhostDirReader or EhostDocReader first, because some Doc extension used to store meta data will not be automatically recreated by loading.
    train_docs, test_docs=joblib.load(pickle_file)

..\data\n2c2\spacy_docs.joblib already exists, load them directly


# Define sampling functions

In [8]:
rounds =10
seed= 14

In [9]:
len(train_docs), len(test_docs)

(303, 202)

### Define CRF Wrapper

In [18]:
from CRFWrapper import spans_to_bio, convert_docs, word2features, sent2features,compute_metrics_and_averages,  CRFModel
from ALSampler import SamplingSimulator, ModelSamplingSimulator, VBSamplingSimulator


In [11]:
faiss_index_path= r'..\data\n2c2\faiss_index_st768'

In [12]:
## Get all annotation types: 
annos=set()
for d in train_docs:
    for anno in d.spans.keys():
        annos.add(anno)
print(annos)

{'Frequency', 'Drug', 'ADE', 'Form', 'Dosage', 'Strength', 'Duration', 'Route', 'Reason'}


In [13]:
pickle_embedding_file= r'..\data\n2c2\embedding_df.joblib'
crf_model=CRFModel(anno_types=annos)
embedding_df=joblib.load(pickle_embedding_file)
r_simulator=VBSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed, faiss_index_path=faiss_index_path, embedding_df=embedding_df)

2024-04-04 22:57:25.610 | DEBUG    | ALSampler:__init__:109 - Loading index...
2024-04-04 22:57:26.153 | DEBUG    | ALSampler:__init__:111 - done


### Test run

In [14]:
scores=r_simulator.simulate_rounds(boostrap_times=3)

2024-04-04 22:57:53.232 | INFO     | ALSampler:simulate_rounds:45 - simulate round 0.
2024-04-04 22:57:53.234 | DEBUG    | ALSampler:keep_sample:35 - The first round sampling will be random
2024-04-04 22:57:53.237 | INFO     | ALSampler:keep_sample:39 - current sampled: 30, remaining: 273
2024-04-04 22:57:57.029 | DEBUG    | CRFWrapper:fit:251 - Reset and train CRF model...
2024-04-04 22:58:53.034 | DEBUG    | CRFWrapper:fit:259 - Training complete.
2024-04-04 22:58:53.133 | DEBUG    | CRFWrapper:bootstrap_eval:280 - Predicting eval docs...
2024-04-04 22:59:29.807 | DEBUG    | CRFWrapper:bootstrap_eval:284 - Calculate scores from bootstrapping 3 times
2024-04-04 22:59:33.816 | DEBUG    | CRFWrapper:bootstrap_eval:293 - complete
2024-04-04 22:59:33.966 | DEBUG    | ALSampler:simulate_rounds:56 - {'precision': 0.9422456488896683, 'recall': 0.7769557525403061, 'f1': 0.8516533835173723}
2024-04-04 22:59:33.968 | INFO     | ALSampler:simulate_rounds:45 - simulate round 1.
2024-04-04 22:59:3

In [15]:
def compute_mean_ci(scores):
    ave=np.mean(scores)
    ci=np.percentile(scores, [2.5, 97.5])
    return ave, ci

summary={'precision': [], 'pl':[], 'pu': [], 'recall': [], 'rl':[], 'ru': [], 'f1':[], 'fl':[], 'fu': []}
for s in scores:    
    for k,v in s.items():
        ave, (l, u)=compute_mean_ci(v)
        summary[k].append(ave)
        summary[k[0]+'l'].append(l)
        summary[k[0]+'u'].append(u)


In [16]:
pd.DataFrame(summary)

,precision,pl,pu,recall,rl,ru,f1,fl,fu
0,0.942246,0.941630,0.942988,0.776956,0.774381,0.780254,0.851653,0.849855,0.853937
1,0.939879,0.939207,0.940350,0.813052,0.809604,0.816221,0.871875,0.869971,0.873879
2,0.939547,0.937581,0.941075,0.822603,0.820527,0.825724,0.877190,0.876260,0.878127
3,0.944308,0.943111,0.946018,0.837092,0.834839,0.839625,0.887472,0.885732,0.888681
4,0.945256,0.944303,0.946406,0.845447,0.843417,0.848065,0.892568,0.891948,0.893600
5,0.941062,0.939423,0.943176,0.850685,0.847509,0.853065,0.893589,0.892786,0.894164
6,0.944856,0.942882,0.946125,0.853192,0.852803,0.853750,0.896687,0.895583,0.897567
7,0.945070,0.943365,0.946529,0.854524,0.850910,0.857363,0.897514,0.896176,0.898310
8,0.945791,0.944488,0.947165,0.856824,0.854951,0.860053,0.899109,0.898077,0.900329
9,0.947499,0.945725,0.949551,0.861023,0.857975,0.863436,0.902193,0.899715,0.904448


## Bootstrap 3 runs

In [17]:
logger.remove()
logger.add(sys.stderr, level='INFO')

1

In [15]:
boostrap_runs=3
total_round=10

In [16]:
random.seed(14)
seeds=[random.randint(1,10000000) for  _ in range(boostrap_runs)]
seeds

[1792286, 8843471, 4142887]

In [17]:
all_scores=[]
embedding_df=joblib.load(pickle_embedding_file)
for si, seed  in enumerate(seeds):
    logger.info(f'start run {si}.')
    pickle_embedding_file= r'..\data\n2c2\embedding_df.joblib'
    crf_model=CRFModel(anno_types=annos)
    v_simulator=VBSamplingSimulator(train_docs, total_round=10, modelWrapper=crf_model, eval_docs=test_docs, init_seed=seed, faiss_index_path=faiss_index_path, embedding_df=embedding_df)
    scores=v_simulator.simulate_rounds(boostrap_times=200)
    all_scores.append(scores) 

2024-03-24 21:53:27.141 | INFO     | __main__:<module>:4 - start run 0.
2024-03-24 21:53:37.286 | INFO     | ALSampler:simulate_rounds:45 - simulate round 0.
2024-03-24 21:53:37.289 | INFO     | ALSampler:keep_sample:39 - current sampled: 30, remaining: 273
2024-03-24 21:59:31.233 | INFO     | ALSampler:simulate_rounds:45 - simulate round 1.
2024-03-24 21:59:31.612 | INFO     | ALSampler:sample_next_round:192 - distance shape: (47530, 10), max to retrieve 47501 sentences
2024-03-24 21:59:33.539 | INFO     | ALSampler:keep_sample:39 - current sampled: 60, remaining: 243
2024-03-24 22:06:38.235 | INFO     | ALSampler:simulate_rounds:45 - simulate round 2.
2024-03-24 22:06:38.764 | INFO     | ALSampler:sample_next_round:192 - distance shape: (47530, 10), max to retrieve 47501 sentences
2024-03-24 22:06:40.669 | INFO     | ALSampler:keep_sample:39 - current sampled: 90, remaining: 213
2024-03-24 22:14:59.070 | INFO     | ALSampler:simulate_rounds:45 - simulate round 3.
2024-03-24 22:14:59.

In [18]:
joblib.dump(all_scores, r'..\data\vb_sampling\ner_vbmax_scores.joblib')

['..\\data\\vb_sampling\\ner_vbmax_scores.joblib']